In [1]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle

In [2]:
import random

In [3]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

typeEchantillon : 
- fixe pour utiliser tailleEchantillon
- variable pour utiliser nombre,increment
- total pour utiliser tout le lexique

In [4]:
typeEchantillon="variable"
tailleEchantillon=15000
nombre=10
increment=15000
freqForme=1000000
if typeEchantillon=="total":
    echantillonPrefix="-total-%s" % (dateheure())
elif typeEchantillon=="fixe":
    echantillonPrefix="-%d-%s" % (tailleEchantillon,dateheure())
else:
    echantillonPrefix="-extend"

In [5]:
nomLexique="/Users/gilles/Copy/Python/phonemisation/bdlexique-PDM-Resync.txt"
bdlexique=pd.read_csv(nomLexique,sep=";",names=["ortho","phono","ext","cs","ms","vs","lexeme","L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"],encoding="utf8")
bdlexique.loc[(bdlexique["ms"]=="ij")&(bdlexique["cs"].isin(["V","K"])),"ms"]="MS"
bdlexique.drop_duplicates(inplace=True,subset=["phono","ext","cs","ms","vs","lexeme","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"])

In [6]:
#bdlexique["freq"]=0.0
filtreLFrantext=(bdlexique["lemmeFrantext"]!="***") 
filtreLFilms=(bdlexique["lemmeFilms"]!="***")
filtreFFrantext=(bdlexique["formeFrantext"]!="***") 
filtreFFilms=(bdlexique["formeFilms"]!="***")
#bdlexique.loc[filtreLFrantext|filtreLFilms,"freq"]=bdlexique[filtreLFrantext]["lemmeFrantext"].astype(float)+bdlexique[filtreLFilms]["lemmeFilms"].astype(float)
bdlexique.loc[filtreFFrantext | filtreFFrantext,"freq"]=bdlexique[filtreFFrantext]["formeFrantext"].astype(float)+bdlexique[filtreFFilms]["formeFilms"].astype(float)

In [7]:
inutilesColonnes=["L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"]
for colonne in inutilesColonnes:
    del bdlexique[colonne]

In [8]:
verbes=bdlexique[bdlexique["cs"].isin(["V","K"])].copy()

In [9]:
verbes.loc[verbes["ms"]=="ij","ms"]="MS"

In [10]:
verbes["prob"]=verbes["freq"]/verbes["freq"].sum()

In [11]:
verbes["ms"]=verbes["ms"].fillna("")
verbes["vs"]=verbes["vs"].fillna("pp")
verbes["case"]=verbes["vs"]+verbes["ms"]

##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [12]:
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

In [13]:
bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [14]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

###Définition des cases
- principales pour le paradigme courant
- secondaires pour les cases rares
- totales pour le tout

In [15]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires
listeCases=casesTotales

In [16]:
#with open("2015-Data/"+lexiquePrefix+'-Verbes.pkl', 'rb') as input:
#    verbes = pickle.load(input)

In [17]:
verbes["phono"]=verbes["phono"].apply(lambda x: recoder(x))

In [18]:
paradigmes=pd.pivot_table(verbes, values='prob', index=['lexeme'], columns=['case'])

In [19]:
frequences=paradigmes.apply(lambda c: c / c.sum() * 100, axis=1)

In [20]:
frequences.to_csv("Frequences.csv", sep='\t', encoding='utf-8')

In [19]:
def cluster_rows(X, mu):
    clusters  = {}
    for x in X:
        bestmukey = min([(i[0], np.linalg.norm(x-mu[i[0]])) \
                    for i in enumerate(mu)], key=lambda t:t[1])[0]
        try:
            clusters[bestmukey].append(x)
        except KeyError:
            clusters[bestmukey] = [x]
    return clusters

In [20]:
def find_row_centers(X, K):
    # Initialize to K random centers
    oldmu = X.sample(n=K)
    mu = X.sample(n=K)
    print (oldmu,mu)
    while not has_converged(mu, oldmu):
        oldmu = mu
        # Assign all points in X to clusters
        clusters = cluster_rows(X, mu)
        # Reevaluate centers
        mu = reevaluate_centers(oldmu, clusters)
    return(mu, clusters)

In [ ]:
result=find_centers(frequences.as_matrix(),50)

In [92]:
frequences.reindex

<bound method DataFrame.reindex of case               ai1P      ai1S      ai2P      ai2S      ai3P       ai3S  \
lexeme                                                                       
abaisser            NaN  0.512048       NaN       NaN  1.234940   7.951807   
abandonner     0.071276  0.641483       NaN  0.036958  0.417096   2.510493   
abasourdir          NaN       NaN       NaN       NaN       NaN        NaN   
abattre             NaN  0.263081       NaN       NaN  0.954886   5.271363   
abcéder             NaN       NaN       NaN       NaN       NaN        NaN   
abdiquer            NaN  1.531729       NaN       NaN       NaN   4.376368   
abecquer            NaN       NaN       NaN       NaN       NaN        NaN   
aberrer             NaN       NaN       NaN       NaN       NaN        NaN   
abhorrer            NaN       NaN       NaN       NaN       NaN        NaN   
abjurer             NaN       NaN       NaN       NaN       NaN        NaN   
ablater             NaN      

In [76]:
import numpy as np

In [28]:
def cluster_points(X, mu):
    clusters  = {}
    for x in X:
        bestmukey = min([(i[0], np.linalg.norm(x-mu[i[0]])) \
                    for i in enumerate(mu)], key=lambda t:t[1])[0]
        try:
            clusters[bestmukey].append(x)
        except KeyError:
            clusters[bestmukey] = [x]
    return clusters

In [27]:
def reevaluate_centers(mu, clusters):
    newmu = []
    keys = sorted(clusters.keys())
    for k in keys:
        newmu.append(np.mean(clusters[k], axis = 0))
    return newmu

In [26]:
def has_converged(mu, oldmu):
    return (set([tuple(a) for a in mu]) == set([tuple(a) for a in oldmu]))

In [22]:
def find_centers(X, K):
    # Initialize to K random centers
    oldmu = random.sample(X, K)
    mu = random.sample(X, K)
    while not has_converged(mu, oldmu):
        oldmu = mu
        # Assign all points in X to clusters
        clusters = cluster_points(X, mu)
        # Reevaluate centers
        mu = reevaluate_centers(oldmu, clusters)
    return(mu, clusters)

In [106]:
import random

def init_board(N):
    X = np.array([(random.uniform(-1, 1), random.uniform(-1, 1)) for i in range(N)])
    return X

In [107]:
def init_board_gauss(N, k):
    n = float(N)/k
    X = []
    for i in range(k):
        c = (random.uniform(-1, 1), random.uniform(-1, 1))
        s = random.uniform(0.05,0.5)
        x = []
        while len(x) < n:
            a, b = np.array([np.random.normal(c[0], s), np.random.normal(c[1], s)])
            # Continue drawing points from the distribution in the range [-1,1]
            if abs(a) < 1 and abs(b) < 1:
                x.append([a,b])
        X.extend(x)
    X = np.array(X)[:N]
    return X

In [108]:
X = init_board_gauss(200,3)
find_centers(X, 3)

([array([ 0.49297651,  0.4925374 ]),
  array([ 0.78089445, -0.67627354]),
  array([-0.12223114, -0.35034434])],
 {0: [array([ 0.07455231,  0.33434731]),
   array([ 0.53533947,  0.08725311]),
   array([ 0.19920577,  0.42042305]),
   array([ 0.64337688,  0.53343532]),
   array([ 0.72785551,  0.15983967]),
   array([ 0.39886602,  0.63918934]),
   array([ 0.84788409,  0.56267031]),
   array([ 0.45734935,  0.69630411]),
   array([ 0.26242485,  0.85873967]),
   array([ 0.51728097,  0.65340105]),
   array([ 0.40838142,  0.17369438]),
   array([ 0.29185663,  0.40340913]),
   array([ 0.48017888,  0.59712903]),
   array([ 0.68610124,  0.79371683]),
   array([ 0.24951292,  0.59590505]),
   array([ 0.92383146,  0.5218242 ]),
   array([-0.09903079,  0.45084126]),
   array([-0.03888673,  0.37451001]),
   array([ 0.20611222,  0.17166502]),
   array([ 0.22749495,  0.66463889]),
   array([ 0.07657076,  0.38637424]),
   array([ 0.17383867,  0.80623367]),
   array([ 0.37923708,  0.25016228]),
   array([ 